# Gray - Scott Model of Reaction - Diffusion

A Reaction-Diffusion Model is a mathematical model which generally describes the interdependent interaction between objects in time and space. They are often used in chemistry to simulate the interaction between various chemical interactions in a given space. The Gray - Scott Model is one such reaction diffusion model.

In the discretized form, the Gray- Scott model is described by a system of differential equations given in the Euler scheme by,
$$ A' = A + (D_A \nabla^2 A - AB^2 + f(1-a))\delta t $$
$$ B' = B + (D_B \nabla^2 B - AB^2 - (k+f)B)\delta t $$

where $A, B$ are the associated substances reacting with one another, $D_A$ is the diffusion coefficient of $A$, $D_B$ is the diffusion coefficient of $B$, $f$ is the feed rate of $A$, $k$ is the kill rate of $B$.



In [ ]:
# Packages Used: gray_scott.jl